#### Census data:
https://www.abs.gov.au/census/find-census-data/datapacks
https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_all_for_AUS_short-header.zip

#### Geodata:
https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3
https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files
https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/ASGS_2021_MAIN_STRUCTURE_GPKG_GDA2020.zip

In [ ]:
import geopandas as gpd
import polars as pl
import polars_st as st
import pyogrio

In [ ]:
# ASGA_FILE_PATH = "data/raw/ASGA/{year}/{region}/"
SA1_FILE_PATH = "/home/andre/git/private/electoralyze/data/raw/ASGA/2021/SA1/SA1_2021_AUST_GDA2020.shp"
VIC_ELECTION = "/home/andre/git/private/electoralyze/data/raw/AEC_boundary/federal/2024/VIC/E_VIC24_region.shp"

In [ ]:
# st.read_file()

In [ ]:
electorate = pyogrio.read_dataframe(VIC_ELECTION)
type(electorate)

In [ ]:
st.GeoDataFrame(electorate[["geometry"]].to_dict("records")).with_columns(
    st.geom("geometry").st.centroid()
).st.to_geopandas().explore()

In [ ]:
st.GeoSeries().st.centroid()

In [ ]:
st.GeoDataFrame

In [ ]:
# electorate.explore()
# pyogrio.read_arrow(SA1_FILE_PATH)

In [ ]:
sa1 = pyogrio.read_dataframe(SA1_FILE_PATH)
sa1
# sa1.explore()

In [ ]:
sa1.head(600).explore()

In [ ]:
sa1.to_crs(4326).tail(600).explore()

In [ ]:
sa1_st = st.GeoDataFrame(
    sa1[["SA1_CODE21", "geometry"]]
    .dropna()
    .to_crs(4326)
    .assign(SA1_CODE21=lambda df: df["SA1_CODE21"].astype(float))
    .to_dict("records")
)
sa1_st

In [ ]:
sa1_st_4326 = sa1_st.with_columns(st.geom("geometry").st.set_srid(4326))

In [ ]:
# sa1_st.write_parquet("sa1_processed.parquet")
# sa1_st.st.write_file("sa1_processed.csv",geometry_type="Polygon")
sa1_gdf: gpd.GeoDataFrame = sa1_st_4326.st.to_geopandas().set_crs(4326)
sa1_gdf.to_file("temp/sa1_processed.shp")
sa1_gdf.simplify(0.0001).to_file("temp/0_0001/sa1_processed.shp")
sa1_gdf.simplify(0.0005).to_file("temp/0_0005/sa1_processed.shp")

In [ ]:
sa1_gdf.head(600).explore()

In [ ]:
sa1_st_ = st.from_geopandas(sa1[["SA1_CODE21", "geometry"]].head(1000).dropna().to_crs(4326)).with_columns(
    st.geom("geometry").st.set_srid(4326)
)
sa1_gdf_ = sa1_st_.st.to_geopandas().set_crs(4326).simplify(0.0006)
sa1_gdf_.head(600).explore()

In [ ]:
sa1_gdf_.head(1000)

In [ ]:
simple = pyogrio.read_dataframe("temp/sa1_processed.shp").simplify(0.1)

In [ ]:
pyogrio.read_dataframe("temp/sa1_processed.shp")["geometry"].loc[0]

In [ ]:
pyogrio.read_dataframe("temp/sa1_processed.shp").loc[0]["geometry"].simplify(0.001).area

In [ ]:
# simple.explore()

In [ ]:
st.GeoDataFrame(pyogrio.read_dataframe("temp/sa1_processed.shp").to_dict("records"))

In [ ]:
sa1_st.head(10).st.to_geopandas().explore()

In [ ]:
sa1_st_simple = sa1_st.with_columns(st.geom("geometry").st.centroid())

In [ ]:
# sa1_st_simple.st.to_geopandas().explore()

In [ ]:
import os

os.environ["PROJ_DEBUG"] = "3"

In [ ]:
st.read_file(VIC_ELECTION).st.to_geopandas()

In [ ]:
st.read_file(SA1_FILE_PATH)